In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import itertools
from collections import OrderedDict
from functools import partial

from sklearn.datasets import load_svmlight_file, make_circles


from links import LinksClassifier
from logit import LogisticRegressionPairwise, LogisticRegression


from sklearn.model_selection import ParameterGrid, StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook as tqdm

from new_experiment_runner.cacher import CSVCacher

In [17]:
%matplotlib tk
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
params = pd.read_csv('data/samping.csv')

In [41]:
datasets =params.dataset.unique()
methods = np.unique(params.sampling_method)




In [76]:
fig, ax = plt.subplots(ncols=len(datasets), nrows=2)
for i_ds, ds in enumerate(datasets):
    ds_scores = params.ix[params['dataset'] == ds]
    gb = ds_scores.groupby('sampling_method')
    ds_by_method = map(lambda kv: kv[1], gb)
    ax[0, i_ds].boxplot(map(lambda d: d['test_score'], ds_by_method), labels=methods.astype(int))
    ax[0, i_ds].set_yticks(np.round(np.linspace(0.5, 1.0, 5), 2))
    ax[0, i_ds].set_ylim(0.5, 1)
    ax[0, i_ds].set_title(ds)
    
    ax[1, i_ds].boxplot(map(lambda d: d['cv_score'], ds_by_method), labels=methods.astype(int))
    ax[1, i_ds].set_yticks(np.round(np.linspace(0.5, 1.0, 5), 2))
    
ax[0, 0].set_ylabel('test score')
ax[1, 0].set_ylabel('cv score')
    

    
    

In [74]:
fig, ax = plt.subplots(ncols=len(datasets), nrows=3)
for i_ds, ds in enumerate(datasets):
    ds_scores = params.ix[params['dataset'] == ds]
    gb = ds_scores.groupby('sampling_method')
    ds_by_method = map(lambda kv: kv[1], gb)
    
    ax[0, i_ds].set_title(ds)
    ax[0, i_ds].boxplot(map(lambda d: d['alpha'], ds_by_method))
    ax[1, i_ds].boxplot(map(lambda d: d['beta'], ds_by_method))
    ax[2, i_ds].boxplot(map(lambda d: d['delta'], ds_by_method), labels=methods.astype(int))
    
ax[0, 0].set_ylabel('alpha')
ax[1, 0].set_ylabel('beta')
ax[2, 0].set_ylabel('delta')
#fig.tight_layout(w_pad=10)
    

In [75]:
fig, ax = plt.subplots(ncols=len(datasets), nrows=3)
for i_ds, ds in enumerate(datasets):
    ds_scores = params.ix[params['dataset'] == ds]
    gb = ds_scores.groupby('sampling_method')
    ds_by_method = map(lambda kv: kv[1], gb)
    
    ax[0, i_ds].set_title(ds)
    ax[0, i_ds].boxplot(map(lambda d: d['alpha/n'], ds_by_method))
    ax[1, i_ds].boxplot(map(lambda d: d['beta/n'], ds_by_method))
    ax[2, i_ds].boxplot(map(lambda d: d['delta/n'], ds_by_method), labels=methods.astype(int))
    
    ax[0, i_ds].set_yticks(np.round(np.linspace(params['alpha/n'].min(), params['alpha/n'].max(), 10), 1))
    ax[1, i_ds].set_yticks(np.round(np.linspace(params['beta/n'].min(), params['beta/n'].max(), 10), 1))
    ax[2, i_ds].set_yticks(np.round(np.linspace(params['delta/n'].min(), params['delta/n'].max(), 10), 1))
ax[0, 0].set_ylabel('alpha/n')
ax[1, 0].set_ylabel('beta/n')
ax[2, 0].set_ylabel('delta/n')
#fig.bbox_inches('tight')

In [50]:
datasets['alpha/n'].max()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [13]:
help(plt.boxplot)

Help on function boxplot in module matplotlib.pyplot:

boxplot(x, notch=None, sym=None, vert=None, whis=None, positions=None, widths=None, patch_artist=None, bootstrap=None, usermedians=None, conf_intervals=None, meanline=None, showmeans=None, showcaps=None, showbox=None, showfliers=None, boxprops=None, labels=None, flierprops=None, medianprops=None, meanprops=None, capprops=None, whiskerprops=None, manage_xticks=True, hold=None, data=None)
    Make a box and whisker plot.
    
    Call signature::
    
      boxplot(self, x, notch=None, sym=None, vert=None, whis=None,
              positions=None, widths=None, patch_artist=False,
              bootstrap=None, usermedians=None, conf_intervals=None,
              meanline=False, showmeans=False, showcaps=True,
              showbox=True, showfliers=True, boxprops=None, labels=None,
              flierprops=None, medianprops=None, meanprops=None,
              capprops=None, whiskerprops=None, manage_xticks=True):
    
    Make a box and